In [5]:
import larq as lq
import tensorflow as tf

In [6]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# Normalize pixel values to be between -1 and 1
train_images, test_images = train_images / 127.5 - 1, test_images / 127.5 - 1

In [13]:
kwargs = dict(input_quantizer="ste_sign",
              kernel_quantizer="ste_sign",
              kernel_constraint="weight_clip")



model = tf.keras.models.Sequential()
model.add(lq.layers.QuantConv2D(32, (3, 3),
                               kernel_quantizer="ste_sign",
                               kernel_constraint="weight_clip",
                               use_bias=False,
                               input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))


model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.BatchNormalization(scale=False))

model.add(lq.layers.QuantConv2D(64, (3, 3), use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Flatten())

model.add(lq.layers.QuantDense(64, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(lq.layers.QuantDense(10, use_bias=False, **kwargs))
model.add(tf.keras.layers.BatchNormalization(scale=False))
model.add(tf.keras.layers.Activation("softmax"))

In [14]:
lq.models.summary(model)

+sequential_5 stats-----------------------------------------------------------------------------------------+
| Layer                   Input prec.           Outputs  # 1-bit  # 32-bit  Memory  1-bit MACs  32-bit MACs |
|                               (bit)                        x 1       x 1    (kB)                          |
+-----------------------------------------------------------------------------------------------------------+
| quant_conv2d_9                    -  (-1, 26, 26, 32)      288         0    0.04           0       194688 |
| max_pooling2d_7                   -  (-1, 13, 13, 32)        0         0       0           0            0 |
| batch_normalization_15            -  (-1, 13, 13, 32)        0        64    0.25           0            0 |
| quant_conv2d_10                   1  (-1, 11, 11, 64)    18432         0    2.25     2230272            0 |
| max_pooling2d_8                   -    (-1, 5, 5, 64)        0         0       0           0            0 |
| batch_no